In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tickers import getTickerList, getCandlePatternList
from datemodule import getDateRange
from indicators import ChannelBreakoutIndicator, SupportResistanceIndicator
from datasource import getFullData
import pandas

In [2]:
import talib
import pandas_ta as ta
from tqdm import tqdm

tickers = getTickerList(all=False)
tickerShortlist = {}
tickerShortlistData = {}

for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    resultDict = {}
    tickerShortlistData[ticker] = {}

    # Fetch Data
    startDate, endDate = getDateRange('1y')
    df = getFullData(ticker, startDate, endDate)
    
    candleIndex = len(df)-1
    
    # RSI
    df['RSI'] = ta.rsi(df.Close, length=14)

    # Candlestick Pattern Recognition
    op = df['Open']
    hi = df['High']
    lo = df['Low']
    cl = df['Close']
    for pattern in getCandlePatternList(all=False):
        sig = getattr(talib, pattern)(op, hi, lo, cl)[candleIndex]
        if sig != 0:
            resultDict[pattern] = sig
    
    # Channel Breakout Indicator Signal
    cbi = ChannelBreakoutIndicator(df, ticker)
    cbi.calculate(40)
    cbSignal = cbi.getBuySell()[-1]
    if (cbSignal != ''):
        resultDict["ChannelBreakoutIndicator"] = cbSignal
        
    # Support Resistance Indicator Signal
    sri = SupportResistanceIndicator(df, 11, 5, ticker)
    sri.calculate()
    srSignal = sri.getBuySell()[-1]
    if (srSignal != ''):
        resultDict["SupportResistanceIndicator"] = srSignal

    # Collect Signals
    if (len(resultDict) != 0):
        resultDict["RSI"] = df['RSI'][candleIndex]
        tickerShortlistData[ticker]["CBI"] = cbi
        tickerShortlistData[ticker]["SRI"] = sri
        tickerShortlist[ticker] = resultDict

  0%|          | 0/3 [00:00<?, ?it/s]/app/indicators/sr_indicator.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['Level'][candleIndex] = cS
 33%|███▎      | 1/3 [00:03<00:07,  3.89s/it]/app/indicators/sr_indicator.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['Level'][candleIndex] = cS
 67%|██████▋   | 2/3 [00:07<00:03,  3.60s/it]/app/indicators/sr_indicator.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [3]:
shortlisted = pandas.DataFrame(tickerShortlist).transpose()
shortlisted["signalCount"] = [row.count() for index, row in shortlisted.iterrows()]
shortlisted = shortlisted.sort_values(by='signalCount', ascending=False).drop(['signalCount'], axis=1)
shortlisted

,CDLHARAMI,ChannelBreakoutIndicator,RSI
RELIANCE,-100,BUY,66.858998


In [ ]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    i = tickerShortlistData[ticker]["CBI"]
    i.showIndicator(len(i.df.index)-1, 40)

{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.85899844556249}
0.9320660390764998 0.9861452597409583


In [5]:
for ticker in shortlisted.index:
    print(tickerShortlist[ticker])
    i = tickerShortlistData[ticker]["SRI"]
    i.showIndicator(len(i.df.index)-1)

{'CDLHARAMI': -100, 'ChannelBreakoutIndicator': 'BUY', 'RSI': 66.85899844556249}
